In [12]:
import numpy as np
import sklearn
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix

In [13]:
# X = np.load("X.npy")
# X = np.load("X_final.npy")
X = np.load("X_final_raw.npy")
X.shape

(289, 646, 60)

In [14]:
X = X.reshape(-1, 646, 60, 1)

In [15]:
# y = (np.hstack((np.zeros(X.shape[0]//2), np.ones(X.shape[0]//2))))
y = (np.hstack((np.zeros(145), np.ones(144))))
y = to_categorical(y)
y.shape

(289, 2)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((202, 646, 60, 1), (87, 646, 60, 1), (202, 2), (87, 2))

In [17]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [18]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [19]:
batch_size = 50
epochs = 15
num_classes = 2

In [20]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(646,60,1),padding='same'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(MaxPooling2D((2, 2),padding='same'))
# model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
# model.add(LeakyReLU(alpha=0.1))                  
# model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model.add(Flatten())
# model.add(Dense(128, activation='linear'))
# model.add(LeakyReLU(alpha=0.1))                  
# model.add(Dense(num_classes, activation='softmax'))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(646,60,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 644, 58, 32)       320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 642, 56, 32)       9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 321, 28, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 319, 26, 64)       18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 317, 24, 64)       36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 158, 12, 64)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 121344)            0         
__________

In [22]:
# , validation_data=(X_val, y_val)c
train = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

Train on 202 samples, validate on 87 samples
Epoch 1/15
202/202 [==============================] - 83s 412ms/step - loss: 2.2304 - acc: 0.5347 - val_loss: 0.4724 - val_acc: 0.8046
Epoch 2/15
150/202 [=====================>........] - ETA: 13s - loss: 0.4802 - acc: 0.7733

KeyboardInterrupt: 

In [ ]:
predicted_classes = model.predict(X_test)

In [ ]:
acc = []
for i in range(len(predicted_classes)):
    acc.append(np.argmax(predicted_classes[i]) == np.argmax(y_test[i]))
np.sum(acc)/len(acc)